In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-1/train.csv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/test.csv /content

In [0]:
import pandas as pd

prefix = '/content/'
test_df = pd.read_csv(prefix + 'test.csv', header=None)
test_df=test_df.drop(index=0)

test_df = pd.DataFrame({
    'id':test_df[0],
    'text': test_df[1]
})

# display 
test_df.head()

In [0]:
!pip install allennlp
!pip install nltk
!pip install tqdm

!pip install -U spacy 
!python -m spacy download en_core_web_sm

In [0]:
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/elmo-constituency-parser-2018.03.14.tar.gz")

START AGAIN

In [0]:
def extract_modal(obj):
    arr = list() # full set of constituents
    modal= list() # only the modal phrases
    past = list() # only the past phrases
    noun = list()
    verb = list()

    def extract(obj, arr):
        
        if isinstance(obj, dict):
          for k, v in obj.items():
            if isinstance(v, (dict, list)):
                extract(v, arr)
              
            if k=='nodeType' and v=='NP':
                arr.append(v)
            if k=='nodeType' and v=='MD':
                arr.append(v)
            if k=='nodeType' and v=='VP':    
                arr.append(v)
            if k=='nodeType' and v=='VBD': # Verb, past tense
                arr.append(v) 
            if k=='nodeType' and v=='ADVP': # Adverb Phrase
                arr.append(v)           
            if k=='nodeType' and v=='RB':   # Adverb 
                arr.append(v)
            if k=='nodeType' and v=='CC':   # Coordinating conjunction
                arr.append(v)
            if k=='nodeType' and v=='IN':   # Preposition or subordinating conjunction 
                arr.append(v)        
            if k=='nodeType' and v=='SBAR': # Clause starting subordinating conjunction
                arr.append(v)
            if k=='nodeType' and v=='SINV': # Inverted declarative sentence, i.e. one in which the subject follows the tensed verb or modal.
                arr.append(v)

        elif isinstance(obj, list):
          for item in obj:
            if isinstance(item, dict):
              if ('would'==item['word'].lower() or 
                  'could'==item['word'].lower() or 
                  'd'==item['word'].lower()
              ):
                phrase = scrap_from(obj, item)
                modal.append(phrase) # phrase on the right from item

              if 'MD' in arr: 
                arr.remove('MD')
                for f in obj: # re-visit each fragment 
                  if f['word'].lower().lstrip()=='\'d':
                    modal.append(obj)
                  if f['word'].lower().lstrip().startswith('would'):
                    modal.append(obj)
                  if f['word'].lower().lstrip().startswith('could'):
                    modal.append(obj)
                  if f['word'].lower().lstrip().startswith('should'):
                    modal.append(obj)
                  if f['word'].lower().lstrip().startswith('might'):
                    modal.append(obj)  

              if 'NP' in arr: 
                arr.remove('NP')
                noun.append(obj)  

              if 'SINV' in arr: # Inverted declarative sentence (eg Had it been...)
                arr.remove('SINV')
                past.append(obj) 

              if 'VP' in arr:
                arr.remove('VP')
                #verb.append(obj) # can subsume 'MD'
                if 'wish' == obj[0]['word'].lower():
                  past.append(obj)  
                if 'should' in item['word'].lower():
                  phrase = scrap_from(obj, item)
                  past.append(phrase)   

              if 'VBD' in arr: 
                arr.remove('VBD')
                if 'were' in obj[0]['word'].lower():
                  past.append(obj)
                if 'was' in obj[0]['word'].lower():
                  past.append(obj)  
                if 'had' in obj[0]['word'].lower():
                  past.append(obj)
          
              if 'RB' in arr: 
                arr.remove('RB')
                if 'even' == obj[0]['word'].lower():
                  past.append(obj)
                     
              if 'IN' in arr: 
                  arr.remove('IN') 
                  if 'in' == obj[0]['word'].lower():
                    past.append(obj)  
                  if 'if' == obj[0]['word'].lower(): # space after 'if'
                    past.append(obj)
                  if 'with' == obj[0]['word'].lower():
                    past.append(obj)
                  if 'without'== obj[0]['word'].lower():
                    past.append(obj)   
                  #if 'for' == obj[0]['word'].lower(): # "for example"
                  #  past.append(obj) 

              if 'CC' in arr: 
                  arr.remove('CC')   
                  if 'but' == obj[0]['word'].lower():
                    past.append(obj)   
                  if 'because' == obj[0]['word'].lower():
                    past.append(obj)  
                  if 'although' == obj[0]['word'].lower():
                    past.append(obj)
            
              #arr.append(item) # uncomment for the full set
              #print(item)

              extract(item, arr)
  
        return arr

    results = extract(obj, arr)

    return modal, past, noun, verb

In [0]:
def scrap_from(obj, item): 
  word = str() 
  prev = str() 
  for f in obj: 
    if f['word'] == item['word']: 
      word = ' ' 
    if len(word) > 0: 
      word = word.strip() 
      if (not (
               word.lower()=='\'m' or word.lower()=='\'ll' or
             # word.lower()=='s' or word.lower()=='d' or # 'you get a D for homework'
              word.lower()=='\'s' or word.lower()=='\'d' or
             # word.lower()=='re' or word.lower()=='ve' or word.lower()=='nt' or
              word.lower()=='\'re' or word.lower()=='\'ve' or word.lower()=='n\'t') and  
          not (
              word.lower() in '.:mkx' and prev[-1:].isnumeric()) and # eg:'10:30am', '3M'
          not (
              prev.lower() in '¥$€£&@({[/---#'  or  # after(eg 'S& P')
              word.lower() in '^~+*&%)}]/---...,;:' # before(eg '% 5 M')
          )
      ): 
        word = word + ' '
      word = word + normilize(f['word'])  
      prev = word 
  return word

In [0]:
import re
regex = '\\\'\s+([^\']+)\s+\\\''
def compact(fragment):
# specify the number of replacements by changing the 4th argument
  result = re.sub(regex, '\''+r'\1'+'\'' , fragment)
  return result

In [0]:
def normilize(fragment):
  r = fragment.lstrip()
  r = r.replace('   ',' ')
  r = r.replace('  ',' ')
  r = r.replace(' ,',',') 
  r = r.replace(' .','.')
  r = r.replace('Mr.','Mr. ')
  r = r.replace('Ms.','Ms. ')
  r = r.replace('Mrs.','Mrs. ')
  r = r.replace('.  ','. ') # Mr.
  r = r.replace('... ','...')
  r = r.replace(' --- ','---')
  r = r.replace(' -- ','--')
  r = r.replace(' - ','-')
  #r = r.replace(': ',':') #50/50
  r = r.replace(' !','!')
  r = r.replace(' ?','?')
  r = r.replace('$ ','$')
  r = r.replace('# ','#')
  r = r.replace(' %','%')
  r = r.replace('( ','(')
  r = r.replace(' )',')')
  r = r.replace(' ;',';')
  r = r.replace(' :',':') 
  r = r.replace(' nt','nt')
  r = r.replace(' (k)','(k)') #401(k)
  r = r.rstrip()
  return compact(r)

In [0]:
def separate_children(obj):
    arr = list()

    def separate(obj, arr):
        
        if isinstance(obj, dict):

            for k, v in obj.items():
              if k != 'children':
                if isinstance(v, (dict, list)):
                    separate(v, arr)
                elif k == 'word':      
                    arr.append(v)

        elif isinstance(obj, list):
            for item in obj:
                separate(item, arr)
         
        return arr

    results = separate(obj, arr)
    return results

In [0]:
def extract_constituents():
 for k, v in prediction.items():
  if (k=='hierplane_tree'):
    for ke, val in v.items():
      if (ke=='root'):
        for key, value in val.items():
          if (key == 'children'):
            return extract_modal(value)  

In [0]:
def deduplicate(past):
  for p1 in past:
    for p2 in past:
      if p2 in p1 and len(p1) > len(p2):
        past.remove(p2)          
  return past         

In [0]:
def remove_from(past, modal, last=1):
  pst = past.copy()

  for m in sorted(modal,key=len): # in order of longer consequents
    for p in reversed(sorted(past,key=len)): # shorter antecedents
      if p.rstrip() in m and len(m) > len(p): # p is substring of m
        if p in past and len(past) > last: 
          past.remove(p)
      elif m.rstrip() in p and len(p) > len(m):# m is substring of p
        if m in modal and len(modal) > last: # can't be left empty!
          modal.remove(m) 

  for p in pst: # fresh copy
    for m in modal:
      if m.rstrip() in p: # both p = m or p > m
        if p in past and len(past) > last:
          past.remove(p) # remove p{ m }

  return past, modal         

In [0]:
def separate_from(md,pa):
  if pa in md and len(md) > len(pa):
    if md.index(pa) > 0 :
      md = md[0:md.index(pa)]
      #print('|'+md+'{...}')
    else:
      md = md.replace(pa,'')
      #print('|{...}'+md)  
  return md        

In [0]:
from nltk.tokenize import word_tokenize

def simplify(constit):
  if isinstance(constit, str):
    word = constit.lstrip()
    word = word.replace(' \'m','\'m')
    #word = word.replace('s \'','s\'') # was 'neurtralized'?
    word = word.replace(' \'s','\'s') # a 'surgical' strike!
    word = word.replace(' \'d','\'d')
    word = word.replace(' \'re','\'re')
    word = word.replace(' \'ve','\'ve')
    word = word.replace(' \'ll','\'ll')
    word = word.replace(' n\'t','n\'t') # what about UPPER()?
    return word
    
  gist = list();
  unique = set() 
  for c in constit:
      p = separate_children(c) # list of c['word']s
      for head in sorted(p,key=len):
        #print(head) # head phrase (few words or one char ',')
        tokenized = word_tokenize(head)
        tokens = set(tokenized) #  without duplicates (eg ',')
        if len(unique.intersection(tokens)) <= len(tokens):
          unique.update(tokens) # without duplicate tokens!!
          gist.extend(tokenized)
  #print(gist)     
  result = str()
  prev = str()
  for word in gist:
    word = word.strip()
    #print(prev,'-->',word)
    if (
        not (
             word.lower()=='\'m' or word.lower()=='\'ll' or
             #word.lower()=='s' or word.lower()=='d' or # no ' ' before! 
             word.lower()=='\'s' or word.lower()=='\'d' or
             #word.lower()=='ve' or word.lower()=='nt' or # 're-election'
             word.lower()=='\'re' or word.lower()=='\'ve' or word=='n\'t'
        ) and
        not (word.lower() in '.:mkx' and prev[-1:].isnumeric()) and # after '10:30am'
        not  
        (
          prev.lower() in '¥$€£&@({[/---#' or  # ' ' after
          word.lower() in '^~+*&%)}]/---...,;:'# or before
        )
    ): 
      result = result + ' '
    prev = word
    result = result + word  
  return result

In [0]:
def defragment(fragments, text):
  result = []
  candid = []
  defrag = []
  prev = str() # previous fragment
  start = 0
  end = -1
  for f in sorted(fragments,key=len):
    r = normilize(f)
    #print(prev+' | '+r)
    try: 
      start = text.index(r)
    except ValueError:
      print('{'+r+'} not in:\n '+text)
  
    if start > end: # assume no overlap with prev (ie separte fragment)
      defrag = prev+r
      index = text.find(defrag) # is it exactly consequitive fragment?
      if index < 0 or index > end: # it separate and non-consequitive 
        end = start + len(r) - 1
        prev = r # (len[0:9]==10)
        candid.append({'labels':0, 'frag':r, 'start':start, 'end':end})
      elif index > 0 and len(prev)+len(r) < len(text): # consequitive
        end = len(prev)+len(r)-1
        prev = text[start:end]
        candid.append({'labels':0, 'frag':defrag, 'start':start, 'end':end})
    elif start < end: # next fragment is located before prev end
      defrag = r+prev
      index = text.find(defrag) # is it exactly preceding prev fragment?
      if index > 0 and index+len(prev)+len(r)-1 == end:
        start = index
        prev = text[start:end]
        candid.append({'labels':0, 'frag':defrag, 'start':start, 'end':end})
      elif start >= 0:
        end = start + len(r) - 1
        prev = r # (len[0:9]==10)
        candid.append({'labels':0, 'frag':r, 'start':start, 'end':end})
        
  if len(candid) == 1:
    candid[0].update(labels = 1) # gold    
  result.extend(candid)

  return result       

In [0]:
import nltk
nltk.download('punkt')

original = "There is a possibility, however, that if one had unlimited resources of money, power, and time, devious plans could be designed: There really were many secret experimental programs like 'MK-Ultra' that used hypnosis and drugs to push mind-control to its limits."

verbs = list()
modal = list()
nounm = list()
past = list() 
nounp = list()
nouns = list()

#texts = original.rsplit(',') # chunck the sentence
#for text in texts:

text = original
text = text.replace('\"','') # replace double quotes
text = text.replace(' --- ','---')
text = text.replace(' -- ','--')
text = text.replace(' - ','-') 

prediction = predictor.predict(sentence=text)
mod, pas, nou, ver = extract_constituents()

for v in ver:
  vr = simplify(v) 
  if vr not in verbs:
    verbs.append(vr)

for u in nou:
  nu = simplify(u) 
  if nu not in nouns:
    nouns.append(nu) 

if len(mod)==0: # if no modal('MD') detected 
  for p in pas:
    pa = simplify(p)
    if pa not in past:
        past.append(pa) 

for m in reversed(sorted(mod,key=len)):
  md = simplify(m) # get one string gist
  if len(md)==0:
    continue
  #print('=======================')  
  #print(md)
  
  for nm in reversed(sorted(nouns,key=len)):  
    if nm in md:   
      nounm.append(nm)
      nouns.remove(nm)

  for p in reversed(sorted(pas,key=len)):
    pa = simplify(p)
    if len(pa)==0:
      continue
    #print('-----------------------')
    #print(pa)

    new = separate_from(md, pa)
    if new != md and len(new) > 0:
       if md in modal: # remove original
          modal.remove(md) 
       md = new 

    new = separate_from(pa, md)
    if new != pa and len(new) > 0:
       if pa in past: # remove original
          past.remove(pa) 
       pa = new 

    # separate NPs originating with past tense      
    for np in reversed(sorted(nouns,key=len)):
      if np in pa:
        nounp.append(np)
        nouns.remove(np)
       
        #new = separate_from(pa, np)
        #if new != pa and len(new) > 0:
        #  if pa in past: # remove original
        #      past.remove(pa) 
        #  pa = new 
      
    if pa not in past and len(pa) > 0:
      past.append(pa)         

  if md not in modal and len(md) > 0:        
    modal.append(md)
     
verbs = deduplicate(verbs)
nounm = deduplicate(nounm)
nounp = deduplicate(nounp)
modal = deduplicate(modal)
past = deduplicate(past)

result = remove_from(modal, past)
modal = result[0]
past = result[1]
result = remove_from(past, modal)
past = result[0]
modal = result[1]

nounm = remove_from(nounm, modal,0)[0]
nounp = remove_from(nounp, past, 0)[0]
nounm = remove_from(nounm, verbs,0)[0]
nounp = remove_from(nounp, verbs,0)[0]

# only after nouns are removed above
modal = remove_from(modal, nounm)[0]
past = remove_from(past, nounp)[0]

past = defragment(past, text)
modal = defragment(modal,text)

print(nouns)
print(verbs)
print(nounm)
print(nounp)
print(past)
print(modal)     

In [0]:
import nltk
nltk.download('punkt')

import pandas as pd

from tqdm import tqdm
from time import sleep

prefix = '/content/'
test_df = pd.read_csv(prefix + 'test.csv', header=None)
test_df = test_df.drop(index=0)
  
test_df = pd.DataFrame({
    'id':test_df[0],
    'text': test_df[1], 
    'labels': ''
})

count = test_df['id'].count()
print(count)

i = 0
j = 0
modal_df= pd.DataFrame(columns=['id','labels','text','start', 'end'])
past_df = pd.DataFrame(columns=['id','labels','text','start', 'end'])
simple_df = pd.DataFrame(columns=['id','labels','text'])

for ind in tqdm(test_df.index): 
  original = test_df['text'][ind]

  verbs = list()
  modal = list()
  nounm = list()
  past = list() 
  nounp = list()
  nouns = list()

  text = original
  text = text.replace('\"','') # replace double quotes
  text = text.replace(' --- ','---')
  text = text.replace(' -- ','--')
  text = text.replace(' - ','-') 
 
  prediction = predictor.predict(sentence=text)
  mod, pas, nou, ver = extract_constituents()

  for v in ver:
    vr = simplify(v) 
    if vr not in verbs:
      verbs.append(vr)

  for u in nou:
    nu = simplify(u) 
    if nu not in nouns:
      nouns.append(nu) 

  if len(mod)==0: # if no modal('MD') detected 
    for p in pas:
      pa = simplify(p)
      if pa not in past:
          past.append(pa) 

  for m in reversed(sorted(mod,key=len)):
    md = simplify(m) # get one string gist
    if len(md)==0:
      continue
    #print('=======================')  
    #print(md)
  
    for nm in reversed(sorted(nouns,key=len)):  
      if nm in md:   
        nounm.append(nm)
        nouns.remove(nm)
        nom = nm
    
    for p in reversed(sorted(pas,key=len)):
      pa = simplify(p)
      if len(pa)==0:
        continue
      #print('-----------------------')
      #print(pa)

      new = separate_from(md, pa)
      if new != md and len(new) > 0:
        if md in modal: # remove original
            modal.remove(md) 
        md = new 

      new = separate_from(pa, md)
      if new != pa and len(new) > 0:
        if pa in past: # remove original
            past.remove(pa) 
        pa = new 

      # separate NPs originating with past tense      
      for np in reversed(sorted(nouns,key=len)):
        if np in pa:
          nounp.append(np)
          nouns.remove(np)
        
          #new = separate_from(pa, np)
          #if new != pa and len(new) > 0:
          #  if pa in past: # remove original
          #      past.remove(pa) 
          #  pa = new 
        
      if pa not in past and len(pa) > 0:
        past.append(pa)         

    if md not in modal and len(md) > 0:        
      modal.append(md)
      
  verbs = deduplicate(verbs)
  nounm = deduplicate(nounm)
  nounp = deduplicate(nounp)
  modal = deduplicate(modal)
  past = deduplicate(past)

  result = remove_from(modal, past)
  modal = result[0]
  past = result[1]
  result = remove_from(past, modal)
  past = result[0]
  modal = result[1]

  nounm = remove_from(nounm, modal,0)[0]
  nounp = remove_from(nounp, past, 0)[0]
  nounm = remove_from(nounm, verbs,0)[0]
  nounp = remove_from(nounp, verbs,0)[0]

  # only after nouns are removed above
  modal = remove_from(modal, nounm)[0]
  past = remove_from(past, nounp)[0]

  past = defragment(past, text)
  modal = defragment(modal,text)
 
  #assert len(modal) > 0 # can be legit (eg: 203585)
  for m in modal:
    m_row = pd.Series(data={
      'id': test_df['id'][ind]+'C'+ str(i),
      'labels': m['labels'],
      'text': m['frag'] ,
      'start': m['start'],
      'end': m['end']
    }, name = str(i) )
    modal_df = modal_df.append(m_row)
    i = i+1

  for p in past:
    p_row = pd.Series(data={
      'id': test_df['id'][ind]+'A'+ str(j),
      'labels': p['labels'],
      'text': p['frag'] ,
      'start': p['start'],
      'end': p['end']
    }, name = str(j) )
    past_df = past_df.append(p_row)   
    j = j+1   

  who = str()
  for n in nounp:
    who = who  +  n + ' / '

  what = str()
  for nn in nounm:
    what = what + ' / ' + nn

  i = 0
  j = 0
  for m in modal:
    if len(past) > 0:
      for p in past:
        labels = str(int(m['labels'])*int(p['labels']))
        simple = who + p['frag'] + ' | ' + m['frag'] + what
        #print(labels, simple)
        s_row = pd.Series(data={
          'id': test_df['id'][ind]+'A'+ str(j) + 'C'+ str(i),
          'labels': labels,
          'text': simple
        }, name = str(j) )
        simple_df = simple_df.append(s_row) 
        j = j+1 
    else:
      s_row = pd.Series(data={
        'id': test_df['id'][ind]+'A'+ str(j) + 'C'+ str(i),
        'labels':m['labels'],
        'text': who + ' | ' + m['frag'] + what
      }, name = str(i) )
      simple_df = simple_df.append(s_row) 

    i = i+1

  sleep(1)  

past_df.to_csv(prefix+'antecedents.tsv', sep='\t', index=False, header=False)
modal_df.to_csv(prefix+'consequents.tsv', sep='\t', index=False, header=False)
simple_df.to_csv(prefix+'simplified.tsv', sep='\t', index=False, header=False)

In [0]:
!mv /content/antecedents.tsv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/
!mv /content/consequents.tsv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/
!mv /content/simplified.tsv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/